In [6]:
import json
import csv
import pickle
import nltk
import string
import random
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import text
from sklearn.manifold import TSNE

from collections import defaultdict
from functools import partial

import gensim
from gensim import corpora
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
punc = set(string.punctuation)
for sw in punc:
    stops.add(sw)

dir_ = r"C:\Users\Broccoli\Documents\Bayes Hack"
hp2014 = pickle.load(open(dir_ + r"\housing_pipeline_2014_comments.pkl", "rb"))
hi2014 = pickle.load(open(dir_ + r"\housing_inventory_2014_comments.pkl", "rb"))
hi2013 = pickle.load(open(dir_ + r"\housing_inventory_2013_comments.pkl", "rb"))
hi2012 = pickle.load(open(dir_ + r"\housing_inventory_2012_comments.pkl", "rb"))

comments = {}
for d in [hp2014,hi2014,hi2013,hi2012]:
    for k,v in d.items():
        comments[k] = v

In [7]:
def iter_comments(comments=comments):
    for prj in comments:
        for article in comments[prj]:
            for c in article['comments']:
                yield ' '.join([''.join(ch for ch in w if ch not in punc).lower() for w in c.split(' ')])

def iter_articles(comments=comments):         
    for prj in comments:
        for article in comments[prj]:
            yield ' '.join([' '.join([''.join(ch for ch in w if ch not in punc).lower() for w in c.split(' ')]) for c in article['comments']])
        
com2prj = []
for prj in comments:
    for article in comments[prj]:
        [com2prj.append(prj) for c in range(0,len(article['comments']))]

In [8]:
def sentiment_pred(sentence):
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank
#    with open(dir_+r'\negative.txt', 'r') as f:
#        negative = set(f.read().split('\n'))
    negative = opinion_lexicon.negative()
    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence) if word not in stops]
    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []
    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif any(n in word for n in negative):
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral
    if pos_words > neg_words:
        return 1
    elif pos_words < neg_words:
        return -1
    elif pos_words == neg_words:
        return 0

In [ ]:
comments = [[word for word in comment.split() if word not in stops] for comment in iter_articles()]
id2word = corpora.Dictionary(comments)
id2word.filter_extremes(no_below=1, no_above=.90)
comment_corpus = [id2word.doc2bow(c) for c in comments]

lda = gensim.models.ldamodel.LdaModel(comment_corpus, id2word=id2word, num_topics=15, passes=10)
#lsi = gensim.models.LsiModel(comment_corpus, id2word=id2word, num_topics=15)


In [ ]:
import re
for t in lda.print_topics(num_topics=15):
    print(re.sub(r'[0-9\*\+\.]', '', t))

In [16]:
reducedDim = 10

def iter4lda(ic=iter_comments()):
    for comment in ic:
        yield ' '.join([w for w in comment.split(' ') if all(sw not in w for sw in ['like','look','just'])])

comment2address = []
tfidf_vectorizer = text.TfidfVectorizer(min_df=2, stop_words='english', strip_accents = 'unicode', lowercase=True, 
                                        ngram_range=(1,2), norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(iter4lda())
words_tfidf = np.array(tfidf_vectorizer.get_feature_names())

tsvd_tfidf = TruncatedSVD(reducedDim)
tsvd_tfidf.fit(tfidf_matrix)
V_tf = tsvd_tfidf.components_
U_tf = tsvd_tfidf.transform(tfidf_matrix)
print(V_tf.shape)
print(U_tf.shape)
print(len(comment2address))

topic_words={}
for i in range(reducedDim):
    row = tsvd_tfidf.components_[i,:]
    argsorted = np.argsort(row)[::-1][:25]
    scores = row[argsorted]
    top_words = words_tfidf[argsorted]
    topic_words[i]=' , '.join(top_words)
for i in topic_words:
    print(i)
    print(topic_words[i])

(10, 71459)
(30905, 10)
0
0
building , parking , people , city , sf , housing , don , new , think , market , lot , good , street , units , really , build , san , buildings , project , area , francisco , san francisco , going , know , want
1
parking , spaces , cars , parking spaces , street , lot , parking lot , car , street parking , space , garage , parking space , transit , spots , bike , building , parking garage , parking spots , congestion , units , retail , ratio , bike parking , unit , spot
2
parking , san , san francisco , francisco , cars , spaces , parking spaces , car , city , street parking , parking lot , street , county , transit , county san , garage , bike , city san , parking space , congestion , spots , parking garage , parking spots , property county , people
3
building , design , new , buildings , francisco , san francisco , san , new building , lot , story , nice , height , planning , proposed , story building , taller , space , love , ugly , corner , tall , street

In [ ]:
count_vectorizer = text.CountVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english', lowercase=True)
count_matrix = count_vectorizer.fit_transform(iter4lda())
words_count = np.array(count_vectorizer.get_feature_names())

tsvd_count = TruncatedSVD(reducedDim)
tsvd_count.fit(count_matrix)
V_tc = tsvd_count.components_
U_tc = tsvd_count.transform(count_matrix)
print(V_tc.shape)
print(U_tc.shape)
print(len(comment2address))

topic_words_count={}
for i in range(reducedDim):
    row = tsvd_count.components_[i,:]
    argsorted = np.argsort(row)[::-1][:25]
    scores = row[argsorted]
    top_words = words_count[argsorted]
    topic_words_count[i]=' , '.join(top_words)
for i in topic_words_count:
    print(i)
    print(topic_words_count[i])

In [ ]:
import random
random.sample([c for c in iter_comments()], 10)

In [ ]:
address2topic = defaultdict(partial(np.ndarray, reducedDim))

row = 0
for address in comment2address:
    address2topic[address] += U_tf[row]
    row += 1

topic2address = {}
for i in range(1,10):
    topic2address[i] = [a for a in address2topic if address2topic[a][i] == max(address2topic[a][1:])]

In [ ]:
for i in range(0,9):
    if address2topic['1078 hampshire st'][i] == max(address2topic['1078 hampshire st']):
        print(topic_words[i])

In [ ]:
prjsent = defaultdict(list)

for prj in random.sample(list(comments),25):
    for article in comments[prj]:
            sentiment = Counter()
            for com in article['comments']:
                sentiment.update([str(sentiment_pred(' '.join([''.join(ch for ch in w if ch not in punc).lower() for w in com.split(' ')])))])
            prjsent[prj].append(sentiment)